<a href="https://colab.research.google.com/github/opp0615/voice-order/blob/master/clova_speech_recognition_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [28]:
from openpyxl import load_workbook
import numpy as np
#가게 이름과 해당 가게의 메뉴 리스트가 담긴 엑셀
tempEx=load_workbook(filename='/content/gdrive/My Drive/Colab Notebooks/stt/store_menu.xlsx')
sheet1=tempEx['Sheet1']
menu=[]

#가게 이름
store_name='서울 카레'

num_list=[['한 ','하나','원'],['두 ','둘','투'],['세 ','셋','쓰리'],['네 ','넷','포'],['다섯 ','다섯','파이브'],['여섯 ','여섯','식스']
          ,['일곱 ','일곱','세븐'],['여덟 ','여덟','에잇'],['아홉 ','아홉','나인'],['열 ','열','텐'],['열하나','열한 '],['열둘','열두 ']
          ,['열셋','열세 '],['열넷','열네 '],['열다섯'],['열여섯'],['열일곱'],['열여덟'],['열아홉'],['스물','스무 ']]

#수량을 묶어 세는 경우를 확인하기 위한 부사 list
all_list=['전부','모두','씩']

#문장이 끊기는 단위를 확인하기 위한 접속사 list
and_list=['고','요']

#해당 음식점의 메뉴를 불러와 전체 메뉴 리스트를 생성
for i in sheet1.rows:
        k=i[0].value
        if k in store_name:
          menu.append(i[1].value)

          
#주문을 mylist에 받아옴          
with open('/content/gdrive/My Drive/Colab Notebooks/stt/test13.txt','rb') as f:
    mylist=[f.read().decode('euc-kr')]
print(mylist)


#menu_list->주문이 들어갈 메뉴들 집합
menu_list={}

#num_index->수량 확인을 위해 메뉴가 말해진 구간을 확정지음
num_index={}

#첫 단어가 음식 이름일 경우
for i in range(0,np.shape(menu)[0]):
  t=menu[i]
  u=mylist[0].find(t)
  if(u==0):
    num_index[t]=[u,u+len(t)]
    menu_list[t]=0
    
#그 이후부터    
for i in range(0,np.shape(menu)[0]):
  t=menu[i]
  u=mylist[0].find(' '+t)
  if(u!=-1):
    menu_list[t]=0
    num_index[t]=[u,u+len(t)+1]
print(menu_list)
print(num_index)

l_menu_list=list(menu_list.keys())
print(l_menu_list)




#메뉴와 메뉴 사이의 구간에서 수량 표현을 찾아 menu_list에 append함
for i in range(len(num_index)):
  flag=0
  for j in range(len(num_list)):
    for w in range(len(num_list[j])):
      if(i+1<len(num_index)):
        result=mylist[0][num_index[l_menu_list[i]][1]:num_index[l_menu_list[i+1]][0]].find(num_list[j][w])
      else:
        result=mylist[0][num_index[l_menu_list[i]][1]:].find(num_list[j][w])
      if(result!=-1):
        menu_list[l_menu_list[i]]=j+1
        flag=1
        break
    
    if(flag==1):
      break
  if(flag==0 and j==len(num_list)-1):
    menu_list[l_menu_list[i]]=0

#수량이 0인 메뉴들을 zero_menu에 append
zero_menu=[]
for i in range(len(menu_list)):
  if(menu_list[l_menu_list[i]]==0):
    zero_menu.append(i)
    
s_index=-9999
e_index=-9999

#부사 표현이 들어간 리스트의 인덱스를 저장
for i in range(len(menu_list)):
  for j in range(len(all_list)):
    a=all_list[j]  
    if(mylist[0].find(a)!=-1):
      e_index=mylist[0].find(a)

#접속사 표현이 들어간 리스트의 인덱스를 저장      
for i in range(len(menu_list)):
  for j in range(len(and_list)):
    a=and_list[j]
    if(mylist[0].find(a)!=-1):
      s_index=mylist[0].find(a)

#부사 표현이 들어간 인덱스와 접속사 표현이 들어간 인덱스들 사이에 있는 메뉴를 revise_list에 저장      
revise_list=[]
for i in range(0,np.shape(menu)[0]):
  t=menu[i]
  u=mylist[0][s_index:e_index+1].find(' '+t)
  if(u!=-1):
    revise_list.append([t])

#revise_list 다음에 나오는 메뉴의 수량을 revise_list의 메뉴들에 적용
for i in range(len(menu_list)):
  if(revise_list[len(revise_list)-1][0]==l_menu_list[i]):
    new_num=menu_list[l_menu_list[i]]
for i in range(len(revise_list)):
  for j in range(len(menu_list)):
    if(revise_list[i][0]==l_menu_list[j]):
      menu_list[l_menu_list[j]]=new_num

#수량표현이 없는 메뉴들의 수량을 0에서 1로 변경      
for i in range(len(menu_list)):
  if(menu_list[l_menu_list[i]]==0):
    menu_list[l_menu_list[i]]=1
  
print(menu_list)    


['토마토하이라이스랑 치킨데리볶음밥 두 개랑 셀리스 화이트로 주세요 밀키스랑 사이다 전부 두 개 주세요']
{'토마토하이라이스': 0, '치킨데리볶음밥': 0, '셀리스 화이트': 0, '밀키스': 0, '사이다': 0}
{'토마토하이라이스': [0, 8], '치킨데리볶음밥': [9, 17], '셀리스 화이트': [22, 30], '밀키스': [35, 39], '사이다': [40, 44]}
['토마토하이라이스', '치킨데리볶음밥', '셀리스 화이트', '밀키스', '사이다']
{'토마토하이라이스': 1, '치킨데리볶음밥': 2, '셀리스 화이트': 1, '밀키스': 2, '사이다': 2}


In [4]:
import sys
import requests

#!pip install pymysql
import pymysql
import os
import sys
sys.path.insert(0, '/content/gdrive/My Drive/Colab Notebooks/stt/')
import config

client_id=config.DATABASE_CONFIG['client_id']
client_secret=config.DATABASE_CONFIG['client_secret']
lang = "Kor" # 언어 코드 ( Kor, Jpn, Eng, Chn )
url = "https://naveropenapi.apigw.ntruss.com/recog/v1/stt?lang=" + lang
data = open('/content/gdrive/My Drive/Colab Notebooks/stt/3.m4a', 'rb')
headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret,
    "Content-Type": "application/octet-stream"
}
response = requests.post(url,  data=data, headers=headers)
rescode = response.status_code
if(rescode == 200):
    print (response.text)
else:
    print("Error : " + response.text)
txt=response.text

     |████████████████████████████████| 51kB 2.0MB/s 
{"text":"파스타는 알리오 올리오나 까르보나라나 라구 파스타 하나 그리고 샐러드는 픽셀로 달려 주시고 그리고 파스타 해산물 토마토 파스타 나도 주세요 그리고 피자는 제일 잘나가는 게 뭐죠 또 만약 비자가 잘 나가는데 클로바 너 피자 하나랑 비스테까 하나 주세요 와인은 화이트 와인 2잔이랑 레드와인 한잔 주세요"}


In [8]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
token=word_tokenize(txt)
tag=nltk.pos_tag(token)
print(token)
print(tag)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
['{', '``', 'text', "''", ':', "''", '파스타는', '알리오', '올리오나', '까르보나라나', '라구', '파스타', '하나', '그리고', '샐러드는', '픽셀로', '달려', '주시고', '그리고', '파스타', '해산물', '토마토', '파스타', '나도', '주세요', '그리고', '피자는', '제일', '잘나가는', '게', '뭐죠', '또', '만약', '비자가', '잘', '나가는데', '클로바', '너', '피자', '하나랑', '비스테까', '하나', '주세요', '와인은', '화이트', '와인', '2잔이랑', '레드와인', '한잔', '주세요', "''", '}']
[('{', '('), ('``', '``'), ('text', 'NN'), ("''", "''"), (':', ':'), ("''", "''"), ('파스타는', 'VBZ'), ('알리오', 'NNP'), ('올리오나', 'NNP'), ('까르보나라나', 'NNP'), ('라구', 'NNP'), ('파스타', 'NNP'), ('하나', 'NNP'), ('그리고', 'NNP'), ('샐러드는', 'NNP'), ('픽셀로', 'NNP'), ('달려', 'NNP'), ('주시고', 'NNP'), ('그리고', 'NNP'), ('파스타', 'NNP'), ('해산물', 'NNP'), ('토마토', 'NNP'), ('파스타', 'NNP'), ('나도', 'NNP'), ('주세요', 'NNP

In [7]:
#!pip install konlpy
from konlpy.tag import Okt
okt=Okt()
print(okt.morphs(txt))


     |████████████████████████████████| 19.4MB 2.6MB/s 
     |████████████████████████████████| 2.7MB 44.2MB/s 


/usr/local/lib/python3.6/dist-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


['{"', 'text', '":"', '파스타', '는', '알리오', '올', '리오', '나', '까르보나라', '나', '라', '구', '파스타', '하나', '그리고', '샐러드', '는', '픽셀', '로', '달려', '주시', '고', '그리고', '파스타', '해산물', '토마토', '파스타', '나도', '주세요', '그리고', '피자', '는', '제일', '잘나가는', '게', '뭐', '죠', '또', '만약', '비자', '가', '잘', '나가는데', '클로', '바', '너', '피자', '하나', '랑', '비스테까', '하나', '주세요', '와인', '은', '화이트', '와인', '2', '잔', '이랑', '레드와인', '한잔', '주세요', '"}']


In [0]:
from openpyxl import load_workbook
import numpy as np
tempEx=load_workbook(filename='/content/gdrive/My Drive/Colab Notebooks/stt/store_menu.xlsx')
sheet1=tempEx['Sheet1']
store_name='서울 카레'

for i in sheet1.rows:
        k=i[0].value
       
        if k in store_name:
          print(i[1].value)

치킨데리볶음밥
토마토하이라이스
2프로 스테이크
셀리스 화이트
소고기사골카레
갈비살떡볶이
모둠버섯카레
독일식 수제 소시지 세트
버드와이저
아메리카노
까르후
콜라
애플 스파클링
밀키스
감바스 알 아히요 세트
펑크IPA
사이다
닭고기버터카레
